
## 1) Required libraries

We import libraries for data handling, training, evaluation, and saving artifacts.


In [ ]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import joblib



## 2) Load dataset

Notes:
- `parse_dates=['date']` converts to `datetime64`, needed for time-based processing and split.  
- Sorting by the datetime index preserves temporal order for lag features and splitting.


In [ ]:
path = 'household.csv'
df = pd.read_csv(path, parse_dates=['date'], low_memory=True)
df = df.set_index('date').sort_index()


df = df.drop(columns=['main','description'], errors='ignore')

print("Dataset shape (rows, cols):", df.shape)
display(df.head())



## 3) Quick EDA

Goal: list useful columns and check missing values for the data description section.


In [ ]:
print("Columns:", df.columns.tolist())
print("\nMissing values per column:")
print(df.isna().sum())
display(df.describe().T[['count','mean','std','min','max']])



## 4) Feature engineering — key features

Explanation of created features:
- `ap_lag_1h`: `active_power` one hour ago — short-term autocorrelation.  
- `ap_lag_24h`: same hour of the previous day — daily cycle.  
- `ap_roll_24h`: 24-hour rolling mean — trend.  
- `hour, weekday, is_weekend`: time-of-day, day-of-week, and weekend indicator.

Technical note: create features on a copy (`data = df.copy()`) to avoid growing `df` when re-running cells.


In [ ]:


data = df.copy()

# Lag features
data['ap_lag_1h'] = data['active_power'].shift(1)
data['ap_lag_24h'] = data['active_power'].shift(24)

# 24h rolling mean (drop NA later)
data['ap_roll_24h'] = data['active_power'].rolling(window=24, min_periods=1).mean()

# Time features
data['hour'] = data.index.hour
data['weekday'] = data.index.weekday
data['is_weekend'] = (data['weekday'] >= 5).astype(int)


keep = ['active_power','ap_lag_1h','ap_lag_24h','ap_roll_24h',
        'temp','feels_like','humidity','pressure','speed',
        'current','voltage','apparent_power','hour','weekday','is_weekend']
keep = [c for c in keep if c in data.columns]

# Drop NA caused by lag/rolling
data = data[keep].dropna().copy()

print("After feature engineering shape (rows, cols):", data.shape)
display(data.head())



## 5) Split dataset — 85% Train / 10% Validate / 5% Test (time-based)

Explanation:
- **Train (85%)**: learn patterns on a large portion to stabilize time-series training.  
- **Validate (10%)**: select hyperparameters and check generalization before the final test.  
- **Test (5%)**: held-out data for the final report.


In [ ]:


n = len(data)
train_end = int(n * 0.85)
val_end = int(n * 0.95)

train = data.iloc[:train_end].copy()
validate = data.iloc[train_end:val_end].copy()
test = data.iloc[val_end:].copy()

print("Split sizes -> train, validate, test:", train.shape, validate.shape, test.shape)



## 6) Prepare X and y for each split

We separate features and target for train/validate/test to use validation independently.


In [ ]:

TARGET = 'active_power'

X_train = train.drop(columns=[TARGET])
y_train = train[TARGET]

X_val = validate.drop(columns=[TARGET])
y_val = validate[TARGET]

X_test = test.drop(columns=[TARGET])
y_test = test[TARGET]

print("Feature count:", X_train.shape[1])



## 7) Scaling (why)

Decision Trees do not require scaling, but we standardize inputs to keep consistency across models and ensure stable GridSearch behavior.


In [ ]:

# Scale: fit on train, transform on val/test
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_val_s = scaler.transform(X_val)
X_test_s = scaler.transform(X_test)

print("Scaling completed. (scaler mean sample of first feature)", scaler.mean_[0] if hasattr(scaler,'mean_') else None)



## 8) Hyperparameter tuning on TRAIN with TimeSeriesSplit

We tune `max_depth` and `min_samples_leaf` to control overfitting. We use `TimeSeriesSplit` (no shuffle). Cross-validation runs only on TRAIN to avoid leakage.


In [ ]:

# GridSearch with TimeSeriesSplit (train only)
model = DecisionTreeRegressor(random_state=42)
param_grid = {'max_depth': [5, 8, 12, None], 'min_samples_leaf': [1, 2, 4]}

tscv = TimeSeriesSplit(n_splits=5)
gscv = GridSearchCV(model, param_grid, scoring='neg_mean_absolute_error', cv=tscv, n_jobs=-1, verbose=1)

gscv.fit(X_train_s, y_train)

print("Best params from CV on train:", gscv.best_params_)
print("Best CV (neg MAE):", gscv.best_score_)
best = gscv.best_estimator_



## 9) Evaluate on VALIDATE

Use the validation set to check whether hyperparameters chosen on TRAIN generalize. If metrics are poor, adjust features or grid.


In [ ]:


y_val_pred = best.predict(X_val_s)

mae_val = mean_absolute_error(y_val, y_val_pred)
rmse_val = mean_squared_error(y_val, y_val_pred, squared=False)
r2_val = r2_score(y_val, y_val_pred)

print(f"Validate MAE: {mae_val:.4f}, RMSE: {rmse_val:.4f}, R2: {r2_val:.4f}")



## 10) Refit final model on Train+Validate and evaluate on TEST

After validation is satisfactory, refit on TRAIN+VALIDATE to leverage all training data, then evaluate on TEST. Refit the scaler on TRAIN+VALIDATE before transforming TEST.


In [ ]:

# Refit scaler and final model on train+validate
X_trainval = pd.concat([X_train, X_val], axis=0)
y_trainval = pd.concat([y_train, y_val], axis=0)

scaler = StandardScaler()
X_trainval_s = scaler.fit_transform(X_trainval)
X_test_s = scaler.transform(X_test)  # transform test with scaler fitted on train+val

final_model = DecisionTreeRegressor(**gscv.best_params_, random_state=42)
final_model.fit(X_trainval_s, y_trainval)


y_test_pred = final_model.predict(X_test_s)

mae_test = mean_absolute_error(y_test, y_test_pred)
rmse_test = mean_squared_error(y_test, y_test_pred, squared=False)
r2_test = r2_score(y_test, y_test_pred)

print(f"Test MAE: {mae_test:.4f}, RMSE: {rmse_test:.4f}, R2: {r2_test:.4f}")



## 11) Feature importance — why

Feature importance explains which variables the model relies on most (e.g., `ap_lag_1h` vs `ap_roll_24h`). Include the top-15 table in the report.


In [ ]:


feat_imp = pd.Series(final_model.feature_importances_, index=X_trainval.columns).sort_values(ascending=False)
display(feat_imp.head(15))

feat_imp.to_csv('/mnt/data/dt_85_10_5_feature_importance.csv')



## 12) Plot Actual vs Predicted (sample) — why a sample

Plotting the full test set can be cluttered; we show the first 300 samples to illustrate prediction quality.


In [ ]:


import matplotlib.pyplot as plt
n_plot = 300
plt.figure(figsize=(12,4))
plt.plot(y_test.values[:n_plot], label='Actual')
plt.plot(y_test_pred[:n_plot], label='Predicted', linestyle='--')
plt.legend()
plt.title('Actual vs Predicted (first {} samples of test)'.format(n_plot))
plt.show()



## 13) Visualize a small subtree (max_depth=3) — interpretability

The full tree can be deep; we plot the first 3 levels for readability. Include this figure in the report.


In [ ]:


plt.figure(figsize=(18,10))
plot_tree(final_model, feature_names=X_trainval.columns, max_depth=3, filled=True, fontsize=8)
plt.show()



## 14) Save artifacts (model, metrics, feature importance)

Saving artifacts enables reproducibility and reporting.


In [ ]:

# Save model and outputs
joblib.dump({'model': final_model, 'scaler': scaler, 'features': X_trainval.columns.tolist()},
            '/mnt/data/dt_85_10_5_final.joblib')

pd.DataFrame({'metric': ['MAE_val','RMSE_val','R2_val','MAE_test','RMSE_test','R2_test'],
              'value': [mae_val, rmse_val, r2_val, mae_test, rmse_test, r2_test]}).to_csv('/mnt/data/dt_85_10_5_metrics.csv', index=False)

print("Saved artifacts to /mnt/data/")
